In [1]:
import pandas as pd

# replace with your path
path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
df = pd.read_csv(path + 'en_lyrics.csv')

In [ ]:
# 370714 rows do not contain \n\n
df[~df['lyrics'].str.contains(r'\n\n', na=False)]

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
14,98 Freestyle,rap,Big L,2000,297788,{},"[Verse 1]\nYo, fuck all the glamours and glitz...",16,en,en,en
58,Im Gettin Money,rap,Fabolous,2008,13055,"{""Freck Billionaire""}",[Verse 1 - Freck Billionaire]\nI'm gettin mone...,49,en,en,en
160,Dreams,rap,Domo Genesis,2010,1674,{},Now if I see it in my dreams\nIs it for real? ...,6354,en,en,en
201,Onslaught,rap,Slaughterhouse,2009,13512,{},"[Verse 1 - Royce Da 5'9""]\nYou dealin with a f...",204,en,en,en
208,A Thugz Prayer,rap,Bizzy Bone,2007,161,{},"[Bizzy Bone]\nI am not John, I am not Jesus\nA...",10534,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
3374128,Date,rap,Ab-Soul,2022,1,{},[Intro]\nUh (Hit-Boy)\nAh\n[Ab-Soul]\nThey wan...,7882469,en,en,en
3374149,Homeopathic Medicine Distributors,misc,Lddbioscience,2022,2,{},LDD Bioscience is one of the top Homeopathic M...,7882580,en,en,en
3374164,BMW Love,rap,Nic Steez,2020,2,{},Maybe I need some rest\nMaybe I need a break\n...,7882702,en,en,en
3374178,Gorgeous,rap,TRV T. Dott,2021,5,{},I'm finna go pour a 4 up\nYou say you bout it ...,7882767,en,en,en


In [3]:
df.groupby('tag').count()['title']

tag
country      86658
misc        140977
pop        1393536
rap         964538
rb          155075
rock        633287
Name: title, dtype: int64

In [ ]:
# split stanzas, take into account chorus/verse number
def split_into_stanzas(row):
    stanzas = row['lyrics'].split('\n\n')  # Split by double newlines
    stanza_entries = []
    for stanza in stanzas:
        stanza_entry = row.to_dict()  # Convert all metadata to a dictionary
        stanza_entry['stanza'] = stanza.strip()  # Add the stanza
        stanza_entries.append(stanza_entry)
    return stanza_entries

# Transform the dataframe
stanza_data = []
for _, row in df.iterrows():
    stanza_data.extend(split_into_stanzas(row))  # Extend with stanza dictionaries

# Create a new dataframe with stanzas
stanza_df = pd.DataFrame(stanza_data)

# Drop the original 'lyrics' column (optional)
stanza_df = stanza_df.drop(columns=['lyrics'])

In [ ]:
# for encoding, sentence_transformers.SentenceTransformer
# seems to be the best fit for the task

# simple explanation: this encodes the stanza as a vector
# of real numbers, which can then be passed to models for
# training and classification

# !IMPORTANT: this is the last step of preprocessing
# for both pipelines